# Text generation with an RNN

### libraries

In [17]:
import tensorflow as tf

import numpy as np
import os
import time

In [18]:
path = "./dataset/shahnameh.txt"

### Read the data

In [19]:
f = open(path, 'rb')
text = f.read().decode(encoding='utf-8')

print(len(text)) # length of text

2653849


In [20]:
print(text[:250])

|به نام خداوند جان و خرد
|کزین برتر اندیشه برنگذرد
|خداوند نام و خداوند جای
|خداوند روزی ده رهنمای
|خداوند کیوان و گردان سپهر
|فروزنده ماه و ناهید و مهر
|ز نام و نشان و گمان برترست
|نگارندهٔ بر شده پیکرست
|به بینندگان آفریننده را
|نبینی مرنجان دو بین


In [21]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

vocab

48 unique characters


['\n',
 ' ',
 '(',
 ')',
 '|',
 '«',
 '»',
 '،',
 '؟',
 'ء',
 'آ',
 'أ',
 'ؤ',
 'ئ',
 'ا',
 'ب',
 'ت',
 'ث',
 'ج',
 'ح',
 'خ',
 'د',
 'ذ',
 'ر',
 'ز',
 'س',
 'ش',
 'ص',
 'ض',
 'ط',
 'ظ',
 'ع',
 'غ',
 'ف',
 'ق',
 'ل',
 'م',
 'ن',
 'ه',
 'و',
 'ٔ',
 'پ',
 'چ',
 'ژ',
 'ک',
 'گ',
 'ی',
 '\u200c']

## Process the text

### Vectorize the text

In [22]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [23]:
char2idx

{'\n': 0,
 ' ': 1,
 '(': 2,
 ')': 3,
 '|': 4,
 '«': 5,
 '»': 6,
 '،': 7,
 '؟': 8,
 'ء': 9,
 'آ': 10,
 'أ': 11,
 'ؤ': 12,
 'ئ': 13,
 'ا': 14,
 'ب': 15,
 'ت': 16,
 'ث': 17,
 'ج': 18,
 'ح': 19,
 'خ': 20,
 'د': 21,
 'ذ': 22,
 'ر': 23,
 'ز': 24,
 'س': 25,
 'ش': 26,
 'ص': 27,
 'ض': 28,
 'ط': 29,
 'ظ': 30,
 'ع': 31,
 'غ': 32,
 'ف': 33,
 'ق': 34,
 'ل': 35,
 'م': 36,
 'ن': 37,
 'ه': 38,
 'و': 39,
 'ٔ': 40,
 'پ': 41,
 'چ': 42,
 'ژ': 43,
 'ک': 44,
 'گ': 45,
 'ی': 46,
 '\u200c': 47}

In [24]:
text_as_int = np.array([char2idx[c] for c in text])
text_as_int.shape

(2653849,)

In [25]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'|به نام خداون' ---- characters mapped to int ---- > [ 4 15 38  1 37 14 36  1 20 21 14 39 37]


### Create training examples and targets

In [26]:
# The maximum length sentence
seq_length = 100

# Create training examples
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(15):
    print(i.numpy(), end = ' = ')
    print(idx2char[i.numpy()])

4 = |
15 = ب
38 = ه
1 =  
37 = ن
14 = ا
36 = م
1 =  
20 = خ
21 = د
14 = ا
39 = و
37 = ن
21 = د
1 =  


In [27]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(3):
    print(repr(''.join(idx2char[item.numpy()])))
    print("\n****"*2)

'|به نام خداوند جان و خرد\n|کزین برتر اندیشه برنگذرد\n|خداوند نام و خداوند جای\n|خداوند روزی ده رهنمای\n|خ'

****
****
'داوند کیوان و گردان سپهر\n|فروزنده ماه و ناهید و مهر\n|ز نام و نشان و گمان برترست\n|نگارندهٔ بر شده پیکر'

****
****
'ست\n|به بینندگان آفریننده را\n|نبینی مرنجان دو بیننده را\n|نیابد بدو نیز اندیشه راه\n|که او برتر از نام و'

****
****


In [28]:
# For each sequence, duplicate and shift it to form the input and target text
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [29]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '|به نام خداوند جان و خرد\n|کزین برتر اندیشه برنگذرد\n|خداوند نام و خداوند جای\n|خداوند روزی ده رهنمای\n|'
Target data: 'به نام خداوند جان و خرد\n|کزین برتر اندیشه برنگذرد\n|خداوند نام و خداوند جای\n|خداوند روزی ده رهنمای\n|خ'


In [30]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:10], target_example[:10])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 4 ('|')
  expected output: 15 ('ب')
Step    1
  input: 15 ('ب')
  expected output: 38 ('ه')
Step    2
  input: 38 ('ه')
  expected output: 1 (' ')
Step    3
  input: 1 (' ')
  expected output: 37 ('ن')
Step    4
  input: 37 ('ن')
  expected output: 14 ('ا')
Step    5
  input: 14 ('ا')
  expected output: 36 ('م')
Step    6
  input: 36 ('م')
  expected output: 1 (' ')
Step    7
  input: 1 (' ')
  expected output: 20 ('خ')
Step    8
  input: 20 ('خ')
  expected output: 21 ('د')
Step    9
  input: 21 ('د')
  expected output: 14 ('ا')


### Create training batches

In [31]:
BATCH_SIZE = 32

dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((32, 100), (32, 100)), types: (tf.int32, tf.int32)>

## Build The Model

In [32]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embedding_dim = 25
# Number of RNN units
rnn_units = 1024

In [33]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(int(rnn_units/2),
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.LSTM(rnn_units,
                    return_sequences=True,
                    stateful=True,
                    recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [34]:
model = build_model(
  vocab_size=len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [35]:
# try model
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model.predict(input_example_batch)
    print(example_batch_predictions.shape, "#(batch_size, sequence_length, vocab_size)")

(32, 100, 48) #(batch_size, sequence_length, vocab_size)


In [36]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (32, None, 25)            1200      
_________________________________________________________________
lstm (LSTM)                  (32, None, 512)           1101824   
_________________________________________________________________
lstm_1 (LSTM)                (32, None, 1024)          6295552   
_________________________________________________________________
dense (Dense)                (32, None, 48)            49200     
Total params: 7,447,776
Trainable params: 7,447,776
Non-trainable params: 0
_________________________________________________________________


In [37]:
# sample from the output distribution, to get actual character indices
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [38]:
sampled_indices

array([ 5,  7,  2, 36, 24, 34, 38, 18, 23, 12, 36,  2, 42,  0,  3, 15, 11,
        3,  4, 27, 30, 36, 24, 26, 34,  7,  6, 31, 42, 13, 29, 25, 29, 36,
       44,  7, 24, 34, 24, 32,  6,  1, 32, 14, 10,  0,  1, 27,  2, 34,  0,
       10, 21,  7,  8, 38, 32, 29, 36,  7, 26, 19,  9,  5,  2, 17, 28, 44,
       38, 11, 13, 25, 47, 46, 32,  6,  8, 26, 38, 40, 15,  8, 39, 15, 43,
        2, 23, 45,  3, 25, 41, 22,  4, 44,  8, 14, 19, 13, 31, 23],
      dtype=int64)

In [39]:
# Decode these to see the text predicted
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '|به نام خداوند جان و خرد\n|کزین برتر اندیشه برنگذرد\n|خداوند نام و خداوند جای\n|خداوند روزی ده رهنمای\n|'

Next Char Predictions: 
 '«،(مزقهجرؤم(چ\n)بأ)|صظمزشق،»عچئطسطمک،زقزغ» غاآ\n ص(ق\nآد،؟هغطم،شحء«(ثضکهأئس\u200cیغ»؟شهٔب؟وبژ(رگ)سپذ|ک؟احئعر'


## Train the model

In [40]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [41]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

### Configure checkpoints

In [42]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

In [43]:
EPOCHS = 30
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30


821/821 [==============================] - 55s 63ms/step - loss: 2.0836 - accuracy: 0.3982
Epoch 2/30
821/821 [==============================] - 53s 64ms/step - loss: 1.4111 - accuracy: 0.5772
Epoch 3/30
821/821 [==============================] - 52s 63ms/step - loss: 1.2942 - accuracy: 0.6080
Epoch 4/30
821/821 [==============================] - 53s 64ms/step - loss: 1.2252 - accuracy: 0.6273
Epoch 5/30
821/821 [==============================] - 54s 65ms/step - loss: 1.1681 - accuracy: 0.6433
Epoch 6/30
821/821 [==============================] - 54s 66ms/step - loss: 1.1159 - accuracy: 0.6585
Epoch 7/30
821/821 [==============================] - 54s 66ms/step - loss: 1.0678 - accuracy: 0.6723
Epoch 8/30
821/821 [==============================] - 54s 66ms/step - loss: 1.0233 - accuracy: 0.6851
Epoch 9/30
821/821 [==============================] - 55s 67ms/step - loss: 0.9828 - accuracy: 0.6967
Epoch 10/30
821/821 [==============================] - 55s 67ms/step - loss: 0.9443 - accurac

## Generate text

In [44]:
tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

### The prediction loop

In [51]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 100

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []


    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [54]:
print(generate_text(model, start_string=u"به نام خداوند"))

به نام خداوند بخت
|که او را به نزد کسان پر ز غم
|برون آمد از بیشه و پتنگ
|زنی تا سوی مهتر اندر دراز
|همی کشته شد 
